# 2048 - Testing Agents

Este notebook permite probar diferentes agentes de forma interactiva.

## Agentes Disponibles:
- **RandomAgent**: Agente aleatorio (baseline)
- **ExpectimaxAgent**: Búsqueda Expectimax (recomendado para 2048)
- **MinimaxAgent**: Búsqueda Minimax con Alpha-Beta Pruning

**Nota**: Para experimentos completos, usa `run_experiments.py`

In [ ]:
from datetime import datetime
from GameBoard import GameBoard
from Agent import Agent
from Random_Agent import RandomAgent

# Importar los nuevos agentes
from Expectimax_Agent import ExpectimaxAgent, ExpectimaxAgentOptimized
from Minimax_Agent import MinimaxAgent, MinimaxAgentOptimized
from Heuristics import WEIGHT_CONFIGS

print("Agentes importados correctamente")
print(f"\nConfiguraciones de heurísticas disponibles: {list(WEIGHT_CONFIGS.keys())}")

In [ ]:
def check_win(board: GameBoard):
    return board.get_max_tile() >= 2048


int_to_string = ['UP', 'DOWN', 'LEFT', 'RIGHT']

## Configuración del Agente

Elige el agente que quieres probar modificando la celda siguiente.

In [ ]:
# ============================================================
# ELIGE TU AGENTE AQUÍ
# ============================================================

# Opción 1: Agente Aleatorio (baseline)
# agent = RandomAgent()

# Opción 2: Expectimax (RECOMENDADO para 2048)
agent = ExpectimaxAgentOptimized(depth=3, weights_config='balanced')

# Opción 3: Expectimax con profundidad mayor (más lento pero mejor)
# agent = ExpectimaxAgentOptimized(depth=4, weights_config='balanced')

# Opción 4: Minimax con Alpha-Beta
# agent = MinimaxAgentOptimized(depth=3, use_alpha_beta=True, weights_config='balanced')

# Opción 5: Expectimax con configuración agresiva
# agent = ExpectimaxAgent(depth=3, weights_config='aggressive')

# Opción 6: Expectimax con configuración defensiva
# agent = ExpectimaxAgent(depth=3, weights_config='defensive')

# Opción 7: Expectimax con pesos personalizados
# custom_weights = {
#     'smoothness': 1.0,
#     'monotonicity': 2.0,
#     'empty_cells': 2.7,
#     'max_position': 0.1,
#     'merge_potential': 0.5,
#     'board_value': 0.1,
#     'corner_strategy': 1.0
# }
# agent = ExpectimaxAgent(depth=3, weights=custom_weights)

print(f"Agente seleccionado: {type(agent).__name__}")
if hasattr(agent, 'depth'):
    print(f"  Profundidad: {agent.depth}")
if hasattr(agent, 'weights'):
    print(f"  Configuración de pesos: {list(agent.weights.keys())}")

## Ejecutar Partida

La siguiente celda ejecuta una partida completa con el agente seleccionado.

In [ ]:
board: GameBoard
board = GameBoard()
done = False
moves = 0
total_nodes_explored = 0

print("="*60)
print(f"Iniciando partida con {type(agent).__name__}")
print("="*60)
board.render()

start = datetime.now()

while not done:
    action = agent.play(board)
    
    # Registrar nodos explorados si está disponible
    if hasattr(agent, 'nodes_explored'):
        total_nodes_explored += agent.nodes_explored
    
    # Mostrar progreso cada 10 movimientos
    if moves % 10 == 0:
        print(f'\n--- Move {moves} ---')
        print(f'Action: {int_to_string[action]}')
        print(f'Max tile: {board.get_max_tile()}')
        if hasattr(agent, 'nodes_explored'):
            print(f'Nodes explored: {agent.nodes_explored}')
    
    done = board.play(action)
    done = done or check_win(board)
    moves += 1
    
    # Limitar movimientos por seguridad (evitar loops infinitos)
    if moves > 3000:
        print("\nADVERTENCIA: Límite de movimientos alcanzado (3000)")
        break

elapsed_time = datetime.now() - start

print("\n" + "="*60)
print("RESULTADO FINAL")
print("="*60)
board.render()

print(f'\nTotal time: {elapsed_time}')
print(f'Total Moves: {moves}')
print(f'Max Tile: {board.get_max_tile()}')
print(f'Final Score: {int(board.grid.sum())}')

if total_nodes_explored > 0:
    print(f'Total Nodes Explored: {total_nodes_explored:,}')
    print(f'Avg Nodes per Move: {total_nodes_explored/moves:.1f}')

print(f'Avg Time per Move: {(elapsed_time.total_seconds()/moves)*1000:.2f} ms')

if check_win(board):
    print("\n¡GANASTE! Alcanzaste 2048")
else:
    print(f"\nGame Over. Max tile alcanzado: {board.get_max_tile()}")

print("="*60)

## Ejecutar Múltiples Partidas

Para obtener estadísticas más confiables, puedes ejecutar múltiples partidas.

In [ ]:
from Experiments import GameExperiment

# Configurar el agente que quieres probar
test_agent = ExpectimaxAgentOptimized(depth=3, weights_config='balanced')

# Ejecutar 5 partidas para prueba rápida
experiment = GameExperiment(test_agent, "Test_Agent", num_games=5)
results_df = experiment.run_experiment(verbose=True)

# Ver estadísticas
print("\nEstadísticas resumidas:")
print(results_df[['max_tile', 'final_score', 'moves', 'time_seconds']].describe())

## Comparar Agentes Rápidamente

Compara diferentes configuraciones lado a lado.

In [ ]:
import pandas as pd

# Definir agentes a comparar
agents_to_compare = [
    (RandomAgent(), "Random"),
    (ExpectimaxAgent(depth=2, weights_config='balanced'), "Expectimax_d2"),
    (ExpectimaxAgent(depth=3, weights_config='balanced'), "Expectimax_d3"),
    (MinimaxAgent(depth=3, use_alpha_beta=True, weights_config='balanced'), "Minimax_d3_AB"),
]

# Ejecutar 3 partidas con cada uno
all_results = []
for agent, name in agents_to_compare:
    print(f"\n{'='*60}")
    print(f"Testing: {name}")
    print('='*60)
    
    experiment = GameExperiment(agent, name, num_games=3)
    df = experiment.run_experiment(verbose=False)
    all_results.append(df)

# Combinar resultados
combined_df = pd.concat(all_results, ignore_index=True)

# Comparación
comparison = combined_df.groupby('agent_name').agg({
    'max_tile': ['mean', 'std'],
    'final_score': 'mean',
    'moves': 'mean',
    'time_seconds': 'mean'
}).round(2)

print("\n" + "="*60)
print("COMPARACIÓN RÁPIDA")
print("="*60)
print(comparison)